In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RetailSalesAnalysis") \
    .getOrCreate()

In [3]:
from google.colab import files
uploaded = files.upload()


Saving Sample - Superstore.csv to Sample - Superstore.csv


In [4]:
# Read CSV using Spark
df_spark = spark.read.csv("Sample - Superstore.csv", header=True, inferSchema=True)
df_spark.show(5)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset 

In [7]:
df_spark.printSchema()
from pyspark.sql.functions import col

# Remove nulls
df_spark = df_spark.dropna(subset=["Sales", "Profit", "Category"])

# Cast to numeric
df_spark = df_spark.withColumn("Sales", col("Sales").cast("float"))
df_spark = df_spark.withColumn("Profit", col("Profit").cast("float"))


root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



In [8]:
df_grouped = df_spark.groupBy("Category").sum("Sales", "Profit")
df_grouped.show()

+---------------+-----------------+------------------+
|       Category|       sum(Sales)|       sum(Profit)|
+---------------+-----------------+------------------+
|Office Supplies|703502.9273704886|120632.87826035172|
|      Furniture|733046.8596462011|19686.427146330476|
|     Technology|835900.0648635626|145388.29595325142|
+---------------+-----------------+------------------+



In [9]:
df_sorted = df_grouped.orderBy("sum(Profit)", ascending=False)
df_sorted.show()

+---------------+-----------------+------------------+
|       Category|       sum(Sales)|       sum(Profit)|
+---------------+-----------------+------------------+
|     Technology|835900.0648635626|145388.29595325142|
|Office Supplies|703502.9273704886|120632.87826035172|
|      Furniture|733046.8596462011|19686.427146330476|
+---------------+-----------------+------------------+



In [10]:
df_sorted.coalesce(1).write.csv("category_summary_output", header=True)